In [2]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [3]:
# CHANGE ME!
TASK_ID = 'changing_mean'
AUTHOR = 'g'


In [4]:
task = Task(
    id=TASK_ID,
    name="First element that doesn't change mean",
    description="Return index of the first element you can remove that does not change the mean of the list by more than 0.1",
    plan=[
        {
        "id": "remove",
        "description": "remove",
    }, {
        "id": "orig",
        "description": "mean of the list",
    }, {
        "id": "new",
        "description": "change the",
    }, 
        {
        "id": "diff",
        "description": "by more than 0.1",
    }, 
        {
        "id": "first",
        "description": "index of the first element",
    }, 
    ],
    sample_input={
       "vals":[{"id":0,"value":1.0},{"id":1,"value":2.0},{"id":2,"value":3.0},{"id":3,"value":1.0}],
    },
    sample_output=1,
    category="Aggregation",
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [5]:
q = replace(prototype,
    language='q',
    source='''diffs: 
{abs avg vals[`value] - avg (vals[`value] where vals[`id] <> x)} 
  each vals[`id];
  
changing_mean: first vals[`id] where diffs < 0.1''').load_plan()
q.execute(task)
q.save()

In [7]:
q.widget(task)

Output()

CodeWidget(program='{"task": "changing_mean", "language": "q", "plan": {}, "source": "diffs: \\n  {abs avg val…

In [7]:
r = replace(prototype,
    language='r',
    implementation='',
    source='''library(data.table)
changing_mean <- function(vals) {
   global_mean <- mean(vals$value)
   setDT(vals)[, 
     mean_change := abs(global_mean - mean(vals[id != .BY, value])), 
     by = id] %>%
     filter(mean_change < 0.1) %>%
     slice(which.min(id)) %>%
     pull(id)
}''').load_plan()
r.execute(task)
r.save()

R[write to console]: data.table 1.13.4 using 1 threads (see ?getDTthreads).  Latest news: r-datatable.com

R[write to console]: **********
This installation of data.table has not detected OpenMP support. It should still work but in single-threaded mode.
This is a Mac. Please read https://mac.r-project.org/openmp/. Please engage with Apple and ask them for support. Check r-datatable.com for updates, and our Mac instructions here: https://github.com/Rdatatable/data.table/wiki/Installation. After several years of many reports of installation problems on Mac, it's time to gingerly point out that there have been no similar problems on Windows or Linux.
**********

R[write to console]: 
Attaching package: ‘data.table’


R[write to console]: The following objects are masked from ‘package:dplyr’:

    between, first, last


R[write to console]: The following object is masked from ‘package:purrr’:

    transpose




In [28]:
r.widget(task)

Output()

CodeWidget(program='{"task": "changing_mean", "language": "r", "plan": {}, "source": "library(data.table)\\nch…

In [8]:
imp = replace(prototype,
    language='python-imperative',
    source='''
def get_mean(ls):
  sum = 0
  for e in ls:
    sum += e['value']
  return sum/len(ls)

def changing_mean(vals):
  start_mean = get_mean(vals)
  for i,elem in enumerate(vals):
    new_ls = vals[:i] + vals[i+1:]
    mean = get_mean(new_ls)
    if abs(mean - start_mean) < 0.1:
      return i
  return None
''').load_plan()
imp.execute(task)
imp.save()

In [9]:
func = replace(prototype,
    language='python-functional',
    source='''
def changing_mean(vals):
  start_mean = sum([l['value'] for l in vals])/len(vals)
  diff = lambda m, sm: abs(m - sm)
    
  for i,elem in enumerate(vals):
    new_ls = [x['value'] for x in vals if x != elem]
    mean = sum(new_ls)/len(new_ls)
    if diff(mean,start_mean) < 0.1:
      return i
  return None
''').load_plan()
func.execute(task)
func.save()

In [10]:
pandas = replace(prototype,
    language='python-pandas',
    source='''
def changing_mean(vals):
  mean = vals.value.mean()
  mean_without = vals.apply(lambda row: vals[vals.id != row.id].value.mean(), axis=1)
  return vals[(mean_without - mean).abs() < 0.1].id.tolist()
''').load_plan()
pandas.execute(task)
pandas.save()

In [11]:
#NOTE: Issue with float conversion
datalog = replace(prototype,
    language='datalog',
    source='''.decl avg_except(I:number, Avg:float)    
avg_except(I, Avg) :- 
  vals(I, _),
  N = sum 1.0 : { vals(J, _), I != J },
  Total = sum V : { vals(J, V), I != J },
  Avg = Total / N.
  
changing_mean(I) :-
  vals(I, _),
  N = sum 1.0 : vals(_, _),
  Total = sum V : vals(_, V),
  Avg = Total / N,
  avg_except(I, AvgExcept),
  ((Avg > AvgExcept, Avg - AvgExcept < 0.1);
   (Avg < AvgExcept, AvgExcept - Avg < 0.1)).''').load_plan()
datalog.execute(task)
datalog.save()

Error: Unable to deduce type for variable I in file program.dl at line 12
changing_mean(I) :-
--------------^-----
Error: Unable to deduce type for variable I in file program.dl at line 13
  vals(I, _),
-------^------
Error: Unable to deduce type for variable I in file program.dl at line 17
  avg_except(I, AvgExcept),
-------------^--------------
3 errors generated, evaluation aborted



CalledProcessError: Command 'souffle -F. -D. program.dl' returned non-zero exit status 1.

In [12]:
#NOTE issue with getting average of dataset without current value
sql = replace(prototype,
    language='sql',
    source='''SELECT id
FROM vals v1
WHERE ABS((
  SELECT AVG(value)
  FROM vals v2
  WHERE v1.id != v2.id
) - (SELECT AVG(value) FROM vals)) < 0.1''').load_plan()
sql.save()

In [10]:
program = datalog
program.widget(task)

Output()

CodeWidget(program='{"task": "changing_mean", "language": "datalog", "plan": {"orig": [{"line": 9, "start": 2,…